In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3
!pip install pytorch-lightning
#!pip install accelerate

import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# import torch

# if torch.cuda.is_available():
#     print("Number of GPUs available:", torch.cuda.device_count())
#     for i in range(torch.cuda.device_count()):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
# else:
#     print("CUDA is not available.")

In [4]:
import unsloth
from unsloth import FastLanguageModel
import torch
#import torchvision
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 34,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # finetune_vision_layers = False,
    # finetune_language_layers = True,
)

Unsloth: Making `model.base_model.model.vision_tower.vision_model.encoder` require gradients


In [6]:
df_mar = pd.read_csv("/teamspace/studios/this_studio/English-to-Marathi.csv")

In [7]:
from datasets import Dataset
# Combine English and Marathi columns into a 'text' column
df_mar["text"] = df_mar["English"] + " || " + df_mar["Marathi"]

# Convert the pandas DataFrame into a Hugging Face Dataset
dataset_mar = Dataset.from_pandas(df_mar)

In [8]:
# Convert the pandas DataFrame into a Hugging Face Dataset
custom_prompt = """Below is a document containing English text and its Marathi translation. 
                   Your task is to understand the language complexity, and learn the marathi 
                   semantic content for future translation from English to Marathi language fluently.

### English:
{}

### Marathi:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  

# Function to format the prompts
def formatting_prompts_func(examples):
    english_texts = examples["English"]
    marathi_texts = examples["Marathi"]
    outputs = [""] * len(english_texts) 

    formatted_texts = []
    for english_text, marathi_text, output in zip(english_texts, marathi_texts, outputs):
        formatted_text = custom_prompt.format(english_text, marathi_text, output) + EOS_TOKEN 
        formatted_texts.append(formatted_text)

    return {"text": formatted_texts}

In [9]:
# # Apply the `map` function to the subset
dataset_mar = dataset_mar.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/3636247 [00:00<?, ? examples/s]

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
from unsloth import is_bfloat16_supported

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move to GPU or CPU as required

# Define trainer with consistent precision and multi-GPU support
trainer = SFTTrainer(
    model=model,  
    tokenizer=tokenizer,
    train_dataset=dataset_mar,  # Processed dataset
    dataset_text_field="text",  # Combined 'text' column
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    packing=False,  # Can make training faster fjor short sequences
    args=TrainingArguments(
        per_device_train_batch_size=8,  # Adjust for memory constraints
        gradient_accumulation_steps=4,  # Manage gradients effectively
        num_train_epochs=3,  # Number of training epochs
        warmup_steps=10,
        max_steps=500,  
        learning_rate=2e-6,  # Optimized learning rate, adjust as needed
        fp16=not is_bfloat16_supported(),  
        bf16=is_bfloat16_supported(),  
        logging_steps=10,
        optim="adamw_8bit",  # Memory-efficient optimizer
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=2802,
        output_dir="outputs",  # Output directory
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/3636247 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,636,247 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 273,825,792/12,000,000,000 (2.28% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,4.440200
20,4.366600
30,4.257100
40,4.212900
50,4.106900
60,4.002400
70,3.858700
80,3.685400
90,3.573100
100,3.426500


In [12]:
peshwa_df = pd.read_csv("/teamspace/studios/this_studio/About-Peshwaii-Complete-Marathi-text.txt", delimiter="\t")

In [13]:
# Convert the pandas DataFrame into a Hugging Face Dataset
dataset_peshwe = Dataset.from_pandas(peshwa_df)

In [14]:
# Convert the pandas DataFrame into a Hugging Face Dataset
custom_prompt = """You are a historian specializing in the Peshwa era. Below is a document in Marathi. 
Your task is to:
1. Memorize all the dates and their historical significance.
2. Understand the cultural and linguistic intricacies of Marathi history.
3. Analyze and explain the events and stories surrounding the Peshwa era.

### Marathi:
{}

### Historical Analysis:
{}"""

EOS_TOKEN = tokenizer.eos_token  # End-of-sequence token

# Function to format the prompts
def formatting_prompts_func(examples):
    marathi_texts = examples["Peshwaai"]
    outputs = [""] * len(marathi_texts)  # Placeholder for historical analysis

    formatted_texts = []
    for marathi_text, output in zip(marathi_texts, outputs):
        formatted_text = custom_prompt.format(marathi_text, output) + EOS_TOKEN
        formatted_texts.append(formatted_text)

    return {"texts": formatted_texts}

In [15]:
# # Apply the `map` function to the subset
dataset_peshwe = dataset_peshwe.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/7446 [00:00<?, ? examples/s]

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
from unsloth import is_bfloat16_supported
# import lightning.pytorch as pl
# from lightning.pytorch.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move to GPU or CPU as required

#Checkpointing logic here.
# check_callback = ModelCheckpoint(
#     dirpath = "model_checkpoint",
#     filename="model-{epoch}-{step}-{global_step}",
#     save_top_k = -1,
#     every_n_train_steps = 50,
# )

# Define trainer with consistent precision and multi-GPU support
trainer = SFTTrainer(
    model=model,  
    tokenizer=tokenizer,
    train_dataset=dataset_peshwe,  # Processed dataset
    dataset_text_field="texts",  # Combined 'text' column
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    packing=False,  # Can make training faster for short sequences
    args = TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        warmup_steps=10,
        max_steps=800,
        learning_rate=5e-6,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        save_steps=50,  # <-- Save every 50 steps
        save_total_limit=5,  # Keep last 5 checkpoints only
        optim="adamw_8bit",
        weight_decay=0.05,
        lr_scheduler_type="cosine",
        seed=2802,
        output_dir="outputs_peshwe_cpt",
        save_strategy="steps",  # Important!
        logging_dir="logs_peshwe_ckpt"
    )
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["texts"]:   0%|          | 0/7446 [00:00<?, ? examples/s]

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,446 | Num Epochs = 4 | Total steps = 800
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 273,825,792/12,000,000,000 (2.28% trained)


Step,Training Loss
10,3.491800
20,3.143900
30,2.782100
40,2.500400
50,2.293200
60,2.038800
70,1.798000
80,1.506900
90,1.298500
100,1.189900


TrainOutput(global_step=800, training_loss=1.1134182286262513, metrics={'train_runtime': 9292.1517, 'train_samples_per_second': 2.755, 'train_steps_per_second': 0.086, 'total_flos': 1.9266685807744512e+17, 'train_loss': 1.1134182286262513})

In [ ]:
# For training after the interruption occurred due to restarting of the studio.
trainer.train(resume_from_checkpoint='provide/path')

In [19]:
from unsloth import FastLanguageModel
import time

# Enable inference mode
FastLanguageModel.for_inference(model)

# Marathi prompt for generation
# Marathi prompt updated
custom_prompt = """तुम्ही एक इतिहासकार आहात आणि तुमचे संशोधन पेशवाई कालखंडावर आहे. 
खाली दिलेल्या उदाहरणांप्रमाणे उत्तर सविस्तर आणि माहितीपूर्ण द्या:

उदाहरण १:
विषय: नाना फडणवीसांचे गुप्त राजकारण
सविस्तर माहिती: नाना फडणवीस हे केवळ पेशव्यांचे विश्वासू नसून त्यांनी 'बारभाई मंडळा'च्या माध्यमातून पेशव्यांची सत्ता अबाधित ठेवण्याचा प्रयत्न केला होता. ...

उदाहरण २:
विषय: माधवराव पेशव्यांचा आरोग्यावर झालेला परिणाम
सविस्तर माहिती: माधवराव पेशवे हे अत्यंत बुद्धिमान होते. मात्र त्यांच्या अल्प वयात मृत्यूचे कारण राजकीय तणाव, घरगुती संघर्ष आणि सातत्याने झालेल्या लढायांमुळे निर्माण झालेले आरोग्याचे बिघाड हे होते...

---

आता खालील विषयावर उत्तर लिहा:

विषय: {}
सविस्तर माहिती:"""


# Input topic (modify as needed)
input_text = "पुण्यातील शनिवार वाड्याचे महत्त्व काय आहे?"

# Tokenize the prompt
inputs = tokenizer(
    [custom_prompt.format(input_text)],  
    padding=True, truncation=True, return_tensors="pt"
).to("cuda")

# Generate output
start_time = time.time()
outputs = model.generate(
    **inputs, 
    max_new_tokens=1024,  
    do_sample=True,  
    temperature=0.7,  
    top_p=0.9,  
    repetition_penalty=1.3,  
    no_repeat_ngram_size=3  
)
end_time = time.time()

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the output
print("\nGenerated Text (Marathi):")
print(generated_text)
print(f"\nInference Time: {end_time - start_time:.4f} seconds")


Generated Text (Marathi):
तुम्ही एक इतिहासकार आहात आणि तुमचे संशोधन पेशवाई कालखंडावर आहे. 
खाली दिलेल्या उदाहरणांप्रमाणे उत्तर सविस्तर आणि माहितीपूर्ण द्या:

उदाहरण १:
विषय: नाना फडणवीसांचे गुप्त राजकारण
सविस्तर माहिती: नाना फडणवीस हे केवळ पेशव्यांचे विश्वासू नसून त्यांनी 'बारभाई मंडळा'च्या माध्यमातून पेशव्यांची सत्ता अबाधित ठेवण्याचा प्रयत्न केला होता. ...

उदाहरण २:
विषय: माधवराव पेशव्यांचा आरोग्यावर झालेला परिणाम
सविस्तर माहिती: माधवराव पेशवे हे अत्यंत बुद्धिमान होते. मात्र त्यांच्या अल्प वयात मृत्यूचे कारण राजकीय तणाव, घरगुती संघर्ष आणि सातत्याने झालेल्या लढायांमुळे निर्माण झालेले आरोग्याचे बिघाड हे होते...

---

आता खालील विषयावर उत्तर लिहा:

विषय: पुण्यातील शनिवार वाड्याचे महत्त्व काय आहे?
सविस्तर माहिती: पुणे शहराची शान म्हणजे शनिवारवाडा! मराठा साम्राज्याची राजधानी म्हणून दिलेले या वाड्याचं महत्व खूप मोठं होतं. १७३२ मध्ये बाळाजी विश्वनाथ भट्ट यांनी रघुनाथरायांसाठी हा सुंदर परिसर निवडला तेव्हापासूनच याची सुरुवात झाली. सुरुवातीला इथे फक्त विश्रामगृह नव्हतं तर श्रीमंत छत्रपती शाहू